In [40]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
from sklearn.tree import export_graphviz

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from datetime import datetime as dt
import sys
import os
main_dir = os.path.abspath('..')
export_path = os.path.join(main_dir,'00_Utilities')
# adding Folder_2 to the system path
sys.path.insert(0, export_path)
from dirFunc import *

import warnings
warnings.filterwarnings('ignore')

In [41]:
ruta_origen = 'Datos'

# De SAP
df = pd.read_csv(ruta_origen + '\\datosPreprocesados.csv')

In [42]:
df

,genTipoComp,genModelo,genEstado,genTipoReparacion,genHh,genNVueltas,genDemora,repNP,repNoKitCant,moVariacion,moDesarmeVariacion,moSalvatajeTrabajoReal,moSalvatajeCantPersonal,genPondExp,genPondNivel,genPondAvance,repYCostos,moYCostos,ttaYCostos,genYCostos
0,MO,797F,DESPACHO,Overhaul,11537.0,3,-4.0,748.0,2443.0,0.106629,-0.075239,44.5,7.0,2359.984472,3.462733,0.000270,353396.34,44808.31,165.93,409120.58
1,MO,797B,DESPACHO,Overhaul,12295.0,2,0.0,1157.0,6846.0,0.217008,-0.251932,0.0,0.0,3602.838086,3.455159,0.000000,412310.59,66509.03,2093.40,511163.02
2,MO,793C,DESPACHO,Overhaul,4364.0,2,0.0,624.0,1412.0,0.227270,0.000000,0.0,0.0,3021.180538,3.407597,0.001039,189422.26,32109.42,1198.09,237829.77
3,MO,797B,DESPACHO,Overhaul,14628.0,1,0.0,1136.0,5451.0,0.182184,-0.230151,33.5,4.0,3532.445381,3.339594,0.000350,367988.14,72564.75,11220.23,484908.72
4,MO,797F,DESPACHO,Overhaul,2931.0,2,343.0,702.0,2345.0,-0.019327,-0.107001,27.5,3.0,2884.110353,3.420335,0.000000,388779.30,50695.76,222.50,449449.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,MA,797B,DESPACHO,Proceso Continuo,14547.0,4,0.0,25.0,21.0,0.064000,-0.036145,13.0,3.0,2024.990000,3.150000,0.010725,76560.07,5858.98,0.00,82419.05
872,MA,797F,DESPACHO,Proceso Continuo,21256.0,2,0.0,35.0,24.0,0.074403,0.000000,11.5,2.0,2243.938567,2.757679,0.015878,64758.38,5722.27,0.00,70609.53
873,MF,797F,DESPACHO,Proceso Continuo,23099.0,3,0.0,17.0,18.0,0.991479,1.857143,16.0,1.0,3592.122807,3.025063,0.009153,82203.88,7792.44,0.00,97211.58
874,CO,797B,DESPACHO,Proceso Continuo,15000.0,4,-5.0,35.0,32.0,0.092466,-0.134021,6.0,1.0,2274.367647,3.156863,0.000000,34945.55,5702.74,0.00,41028.49


In [43]:
compCol = 'genTipoComp'
componente = 'DI'

varExplicativas = ['genModelo','genHh','genNVueltas','repNP','moSalvatajeCantPersonal','genPondExp','genPondNivel','genPondAvance']
varPredictiva = ['repNoKitCant']


# Create train test set
filterDf = df[df[compCol] == componente]
train_set, test_set = train_test_split(filterDf, test_size=0.2,random_state=42)
train_set.drop(columns=[compCol],inplace=True)
test_set.drop(columns=[compCol],inplace=True)



dfX = train_set.copy()[varExplicativas]
dfY = train_set.copy()[varPredictiva]

dfX_test = test_set.copy()[varExplicativas]
dfY_test = test_set.copy()[varPredictiva]

catCol = dfX.columns[(dfX.applymap(type) == str).all(0)].tolist()

# Variables Categoricas VS Numericas
numCol = dfX.drop(columns=catCol).columns.tolist()

# Normalizar y añadir variables numericas
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    # ('std_scaler', StandardScaler()),
])
full_pipeline = ColumnTransformer([
("num", num_pipeline, numCol),
("cat", OneHotEncoder(), catCol),
])
df_prepared = full_pipeline.fit_transform(dfX) 
df_prepared_test = full_pipeline.transform(dfX_test)

# Grid Search Execution, calcula tiempo de ejecucion
start = dt.now()

param_grid = [{'n_estimators': [30,40,50], 
                'max_features': [6,8,10],
                'max_samples': [0.5,0.6,0.7,0.8],
                'max_depth':[5],
                'max_leaf_nodes': [6,10],
                'min_samples_leaf':[5]}]
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5,scoring='neg_mean_squared_error',
return_train_score=True)
grid_search.fit(df_prepared, dfY)

running_mins =round((dt.now()-start).seconds/60,2)

feature_importances = grid_search.best_estimator_.feature_importances_
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(np.concatenate(cat_encoder.categories_,axis=0))
attributes = numCol + cat_one_hot_attribs 

features = sorted(zip(feature_importances, attributes), reverse=True)

# Creacion de graficos
# plotFeatures(features,10,filtro,componente)
# treePlot(gridModel, attributes,filtro,componente)

# Calculo de errores
final_predictions = grid_search.predict(df_prepared_test)
final_mse = mean_squared_error(dfY_test, final_predictions)
final_rmse = np.sqrt(final_mse)

r2 = r2_score(y_true  = dfY_test,y_pred  = final_predictions)
final_mae = mean_absolute_error(y_true=dfY_test,y_pred=final_predictions)

# Documentacion de resultados
# Texto de variables
strVar = '\n'.join([f"{i+1}. {el}: {round(imp,3)}" for i,(imp,el) in enumerate(features)]).strip()

# Texto de parametros usados
strParams = "\n".join("{}:\t{}".format(k, v) for k, v in grid_search.best_params_.items()).strip()

In [44]:
print(strVar)

1. repNP: 0.471
2. genPondNivel: 0.208
3. genPondExp: 0.16
4. genHh: 0.082
5. 797F: 0.024
6. genPondAvance: 0.023
7. 854K: 0.015
8. 797B: 0.011
9. genNVueltas: 0.004
10. moSalvatajeCantPersonal: 0.002
11. 854G: 0.0
12. 793F: 0.0
13. 793C: 0.0


In [45]:
print(strParams)

max_depth:	5
max_features:	6
max_leaf_nodes:	10
max_samples:	0.8
min_samples_leaf:	5
n_estimators:	30


In [53]:
media = filterDf[varPredictiva].mean()[0]
print(f'Media: {media}')
print(final_rmse)
print(r2)
print(final_mae)

Media: 83.32116788321167
46.90472590074996
-2.106776269023583
34.16958145827852
